In [1]:
import requests
import json
import requests
import wikipedia as wp
from pprint import pprint
import pandas as pd
import gmaps
import matplotlib.pyplot as plt
from config import g_key




In [2]:
#https://en.wikipedia.org/wiki/List_of_hospitals_in_Georgia_(U.S._state)

def hospital_data():
    html = wp.page("List_of_hospitals_in_Georgia_(U.S._state)").html().encode("UTF-8")
    try: 
        df = pd.read_html(html)[1]  # Try 2nd table first as most pages contain contents table first
    except IndexError:
        df = pd.read_html(html)[0]
    return df


In [3]:
hospital_df= hospital_data()

hospital_df.tail()

,Hospital name,City,County,Acute Care Beds,Traumacenterdesignation,COTH[2]TeachingHospital,YearFounded,Affiliation,Notes
132,WellStar Paulding Hospital,Dallas,Paulding,216,NaN,NaN,1954,WellStar,NaN
133,WellStar Spalding Regional Hospital,Griffin,Spalding,160,NaN,NaN,NaN,WellStar,Formerly Spalding Regional Medical Center
134,WellStar Sylvan Grove Hospital,Jackson,Butts,21,NaN,NaN,NaN,WellStar,NaN
135,WellStar West Georgia Medical Center,LaGrange,Troup,542,Level III,NaN,1937,WellStar,NaN
136,Wills Memorial Hospital,Washington,Wilkes,25,NaN,NaN,1924,â,NaN


In [4]:
# url = "http://www.communitybenefitinsight.org/api/get_hospitals.php?"
# state= 'GA'
# query_url = f"{url}state={state}?"


# response = requests.get(query_url, headers={"User-Agent": "XY"}).json()

# pprint(len(response))
# #88

In [5]:
#Get Place ID of each hospital (https://developers.google.com/places/web-service/search)
hospital_df["Place ID"] = ''

params = {"keyword" : "Hospital",
          "key" : g_key,
          "inputtype" : "textquery"
         }


for index, row in hospital_df.iterrows():
    try:
        url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
        params["input"] = row["Hospital name"]
        response = requests.get(url, params=params).json()
        
        #Get Place ID
        place_id = response["candidates"][0]["place_id"]
        hospital_df.loc[index,"Place ID"] = place_id
    except:
        pass

        

In [6]:
hospital_df.head()

,Hospital name,City,County,Acute Care Beds,Traumacenterdesignation,COTH[2]TeachingHospital,YearFounded,Affiliation,Notes,Place ID
0,AdventHealth Gordon,Calhoun,Gordon,69,NaN,NaN,NaN,AdventHealth,Formerly Gordon Hospital,ChIJfVjpjer9X4gRGK7NwGG_tbg
1,AdventHealth Murray,Chatsworth,Murray,29,NaN,NaN,NaN,AdventHealth,Formerly Murray Medical Center,ChIJ-U0scjndX4gRG-FoXnNaHI4
2,Archbold Memorial Hospital,Thomasville,Thomas,264,Level II,NaN,1925,Archbold,NaN,ChIJsxblg8q17YgR8XXHSBDuObQ
3,Archbold - Brooks County Hospital,Quitman,Brooks,25,NaN,NaN,NaN,Archbold,NaN,ChIJiyuklBp_7ogRmoGX5XNztMM
4,Archbold - Grady General Hospital,Cairo,Grady,60,NaN,NaN,1960,Archbold,NaN,ChIJL2eo1Vup7YgRbnlST-utsVE


In [7]:
#Use place_id (https://developers.google.com/places/web-service/details) to pull in remaining address information
hospital_df["Latitude"] = ''
hospital_df["Longtitude"] = ''
hospital_df["Street Address"] = ''
hospital_df["City"] = ''
hospital_df["County"] = ''
hospital_df["State"] = ''
hospital_df["Zip Code"] = ''
hospital_df["Full Address"] = ''


params = {"key" : g_key
         }


for index, row in hospital_df.iterrows():
    try:
        url = "https://maps.googleapis.com/maps/api/place/details/json"
        params["place_id"] = row["Place ID"]
        response = requests.get(url, params=params).json()
        
        #Pulling new values
        lat = response['result']['geometry']['location']['lat']
        long = response['result']['geometry']['location']['lng']
        street_number = response['result']['address_components'][0]['long_name']
        street_name = response['result']['address_components'][1]['long_name']
        address = f"{street_number} {street_name}"

        city = response['result']['address_components'][2]['long_name']
        county = response['result']['address_components'][3]['long_name']
        state = response['result']['address_components'][4]['long_name']
        zip_code = response['result']['address_components'][6]['long_name']
        full_address = response['result']['formatted_address']
        
        #Putting new values in Hospital DF
        hospital_df.loc[index,"Latitude"] = lat
        hospital_df.loc[index,"Longtitude"] = long
        hospital_df.loc[index,"Street Address"] = address
        hospital_df.loc[index,"City"] = city
        hospital_df.loc[index,"County"] = county
        hospital_df.loc[index,"State"] = state
        hospital_df.loc[index,"Zip Code"] = zip_code
        hospital_df.loc[index,"Full Address"] = full_address
        
    except:
        pass

In [8]:
hospital_df

,Hospital name,City,County,Acute Care Beds,Traumacenterdesignation,COTH[2]TeachingHospital,YearFounded,Affiliation,Notes,Place ID,Latitude,Longtitude,Street Address,State,Zip Code,Full Address
0,AdventHealth Gordon,Calhoun,Gordon County,69,NaN,NaN,NaN,AdventHealth,Formerly Gordon Hospital,ChIJfVjpjer9X4gRGK7NwGG_tbg,34.5102,-84.9263,1035 Red Bud Road Northeast,Georgia,30701,"1035 Red Bud Rd NE, Calhoun, GA 30701, USA"
1,AdventHealth Murray,Chatsworth,Murray County,29,NaN,NaN,NaN,AdventHealth,Formerly Murray Medical Center,ChIJ-U0scjndX4gRG-FoXnNaHI4,34.78,-84.783,707 Old Dalton Ellijay Road,Georgia,30705,"707 Old Dalton Ellijay Rd, Chatsworth, GA 3070..."
2,Archbold Memorial Hospital,Thomasville,Thomas County,264,Level II,NaN,1925,Archbold,NaN,ChIJsxblg8q17YgR8XXHSBDuObQ,30.8252,-83.9721,915 Gordon Avenue,Georgia,31792,"915 Gordon Ave, Thomasville, GA 31792, USA"
3,Archbold - Brooks County Hospital,Quitman,Brooks County,25,NaN,NaN,NaN,Archbold,NaN,ChIJiyuklBp_7ogRmoGX5XNztMM,30.795,-83.5591,903 North Court Street,Georgia,31643,"903 N Court St, Quitman, GA 31643, USA"
4,Archbold - Grady General Hospital,Cairo,Grady County,60,NaN,NaN,1960,Archbold,NaN,ChIJL2eo1Vup7YgRbnlST-utsVE,30.8638,-84.1991,1155 5th Street Southeast,Georgia,39828,"1155 5th St SE, Cairo, GA 39828, USA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,WellStar Paulding Hospital,Hiram,Paulding County,216,NaN,NaN,1954,WellStar,NaN,ChIJMzJn-How9YgRhhfE1kXnu30,33.9011,-84.7816,2518 Jimmy Lee Smith Parkway,Georgia,30141,"2518 Jimmy Lee Smith Pkwy, Hiram, GA 30141, USA"
133,WellStar Spalding Regional Hospital,Griffin,Spalding County,160,NaN,NaN,NaN,WellStar,Formerly Spalding Regional Medical Center,ChIJyexMgeBh9IgRxUd1HX227ns,33.2393,-84.2673,601 South 8th Street,Georgia,30224,"601 S 8th St, Griffin, GA 30224, USA"
134,WellStar Sylvan Grove Hospital,Jackson,Butts County,21,NaN,NaN,NaN,WellStar,NaN,ChIJB5o2d9JB9IgRr-4NjnGt52c,33.3038,-83.9778,1050 McDonough Road,Georgia,30233,"1050 McDonough Rd, Jackson, GA 30233, USA"
135,WellStar West Georgia Medical Center,LaGrange,Troup County,542,Level III,NaN,1937,WellStar,NaN,ChIJ40ZJ5vtWi4gRK6PnDb-BAPU,33.0309,-85.0586,1514 Vernon Road,Georgia,30240,"1514 Vernon Rd, LaGrange, GA 30240, USA"


In [9]:
#Strip out hospitals with correct data
correct_hospitals = hospital_df.loc[hospital_df["State"] == "Georgia"]
correct_hospitals

,Hospital name,City,County,Acute Care Beds,Traumacenterdesignation,COTH[2]TeachingHospital,YearFounded,Affiliation,Notes,Place ID,Latitude,Longtitude,Street Address,State,Zip Code,Full Address
0,AdventHealth Gordon,Calhoun,Gordon County,69,NaN,NaN,NaN,AdventHealth,Formerly Gordon Hospital,ChIJfVjpjer9X4gRGK7NwGG_tbg,34.5102,-84.9263,1035 Red Bud Road Northeast,Georgia,30701,"1035 Red Bud Rd NE, Calhoun, GA 30701, USA"
1,AdventHealth Murray,Chatsworth,Murray County,29,NaN,NaN,NaN,AdventHealth,Formerly Murray Medical Center,ChIJ-U0scjndX4gRG-FoXnNaHI4,34.78,-84.783,707 Old Dalton Ellijay Road,Georgia,30705,"707 Old Dalton Ellijay Rd, Chatsworth, GA 3070..."
2,Archbold Memorial Hospital,Thomasville,Thomas County,264,Level II,NaN,1925,Archbold,NaN,ChIJsxblg8q17YgR8XXHSBDuObQ,30.8252,-83.9721,915 Gordon Avenue,Georgia,31792,"915 Gordon Ave, Thomasville, GA 31792, USA"
3,Archbold - Brooks County Hospital,Quitman,Brooks County,25,NaN,NaN,NaN,Archbold,NaN,ChIJiyuklBp_7ogRmoGX5XNztMM,30.795,-83.5591,903 North Court Street,Georgia,31643,"903 N Court St, Quitman, GA 31643, USA"
4,Archbold - Grady General Hospital,Cairo,Grady County,60,NaN,NaN,1960,Archbold,NaN,ChIJL2eo1Vup7YgRbnlST-utsVE,30.8638,-84.1991,1155 5th Street Southeast,Georgia,39828,"1155 5th St SE, Cairo, GA 39828, USA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,WellStar Paulding Hospital,Hiram,Paulding County,216,NaN,NaN,1954,WellStar,NaN,ChIJMzJn-How9YgRhhfE1kXnu30,33.9011,-84.7816,2518 Jimmy Lee Smith Parkway,Georgia,30141,"2518 Jimmy Lee Smith Pkwy, Hiram, GA 30141, USA"
133,WellStar Spalding Regional Hospital,Griffin,Spalding County,160,NaN,NaN,NaN,WellStar,Formerly Spalding Regional Medical Center,ChIJyexMgeBh9IgRxUd1HX227ns,33.2393,-84.2673,601 South 8th Street,Georgia,30224,"601 S 8th St, Griffin, GA 30224, USA"
134,WellStar Sylvan Grove Hospital,Jackson,Butts County,21,NaN,NaN,NaN,WellStar,NaN,ChIJB5o2d9JB9IgRr-4NjnGt52c,33.3038,-83.9778,1050 McDonough Road,Georgia,30233,"1050 McDonough Rd, Jackson, GA 30233, USA"
135,WellStar West Georgia Medical Center,LaGrange,Troup County,542,Level III,NaN,1937,WellStar,NaN,ChIJ40ZJ5vtWi4gRK6PnDb-BAPU,33.0309,-85.0586,1514 Vernon Road,Georgia,30240,"1514 Vernon Rd, LaGrange, GA 30240, USA"


In [10]:
#strip out hospitals with shifted/bad data
fix_hospitals = hospital_df.loc[hospital_df["State"] != "Georgia"]
fix_hospitals



,Hospital name,City,County,Acute Care Beds,Traumacenterdesignation,COTH[2]TeachingHospital,YearFounded,Affiliation,Notes,Place ID,Latitude,Longtitude,Street Address,State,Zip Code,Full Address
6,Augusta University Children's Hospital of Georgia,Medical District,Augusta,154,NaN,No,NaN,Augusta University,NaN,ChIJoyV_F0jM-YgRqAe60-Abauk,33.4711,-81.9878,1446 Harper Street,Richmond County,United States,"1446 Harper St, Augusta, GA 30912, USA"
7,Augusta University Medical Center,Medical District,Augusta,478,Level I,Yes,NaN,Augusta University,NaN,ChIJyX-W70nM-YgRtaRCmY78n_g,33.4713,-81.9879,1120 15th Street,Richmond County,United States,"1120 15th St, Augusta, GA 30912, USA"
12,Candler Hospital,Midtown,Savannah,384,NaN,NaN,1804,SJCHS,"Oldest hospital in Georgia, and second oldest ...",ChIJpb2fD5N1-4gRM-PXE90FV6c,32.0278,-81.1001,5353 Reynolds Street,Chatham County,United States,"5353 Reynolds St, Savannah, GA 31405, USA"
17,Children's Healthcare of Atlanta Hughes Spalding,Downtown Atlanta,Atlanta,NaN,NaN,Yes,1952,CHOA,NaN,ChIJE0RlP4kD9YgRf-FdJvSfWXw,33.7539,-84.382,35 Jesse Hill Junior Drive Southeast,Fulton County,United States,"35 Jesse Hill Jr Dr SE, Atlanta, GA 30303, USA"
18,Children's Healthcare of Atlanta Scottish Rite,Perimeter Center,Atlanta,NaN,Level II Peds,Yes,1915,CHOA,NaN,ChIJV-6vqjQJ9YgRhvQE1FyLn-g,33.9062,-84.3532,1001 Johnson Ferry Road Northeast,Fulton County,United States,"1001 Johnson Ferry Rd NE, Atlanta, GA 30342, USA"
30,Eastside Medical Center,Peachtree Parkway,Peachtree Corners,249,NaN,NaN,NaN,HCA,NaN,ChIJaTUBHwOh9YgRoHi7L2zoz4E,33.964,-84.2259,100 5707,Gwinnett County,United States,"5707 Peachtree Pkwy #100, Peachtree Corners, G..."
37,Emory Saint Joseph's Hospital,Perimeter Center,Atlanta,276,NaN,NaN,1880,Emory,Formerly Saint Joseph's Hospital of Atlanta,ChIJW3epPTQJ9YgRxf9a5U20qNg,33.9084,-84.3499,5665 Peachtree Dunwoody Road,Fulton County,United States,"5665 Peachtree Dunwoody Rd, Atlanta, GA 30342,..."
39,Emory University Hospital Midtown,SoNo,Atlanta,511,NaN,Yes,1908,Emory,Formerly Crawford W. Long Memorial Hospital,ChIJ42AWonoE9YgRonLLiSdYxtA,33.7687,-84.3864,550 Peachtree Street Northeast,Fulton County,United States,"550 Peachtree St NE, Atlanta, GA 30308, USA"
43,Fannin Regional Hospital,Old Highway 5,Blue Ridge,50,NaN,NaN,NaN,â,NaN,ChIJN7SfeV2vX4gRu5N9mgngcXo,34.9177,-84.3743,North 2855,Fannin County,United States,"2855 Old Hwy 5 North, Blue Ridge, GA 30513, USA"
47,Grady Memorial Hospital,Downtown Atlanta,Atlanta,961,Level I,Yes,1892,â,NaN,ChIJ6Rc8z44D9YgR28vCVjqbUGw,33.7518,-84.382,80 Jesse Hill Junior Drive Southeast,Fulton County,United States,"80 Jesse Hill Jr Dr SE, Atlanta, GA 30303, USA"


In [60]:
#Correct the bad address data for bad hospitals by shifting indices in the for loop
fix_hospitals["Latitude"] = ''
fix_hospitals["Longtitude"] = ''
fix_hospitals["Street Address"] = ''
fix_hospitals["City"] = ''
fix_hospitals["County"] = ''
fix_hospitals["State"] = ''
fix_hospitals["Zip Code"] = ''
fix_hospitals["Full Address"] = ''


params = {"key" : g_key
         }


for index, row in fix_hospitals.iterrows():
    try:
        url = "https://maps.googleapis.com/maps/api/place/details/json"
        params["place_id"] = row["Place ID"]
        response = requests.get(url, params=params).json()
        lat = response['result']['geometry']['location']['lat']
        long = response['result']['geometry']['location']['lng']
        street_number = response['result']['address_components'][1]['long_name']
        street_name = response['result']['address_components'][2]['long_name']
        address = f"{street_number} {street_name}"


        city = response['result']['address_components'][3]['long_name']
        county = response['result']['address_components'][4]['long_name']
        state = response['result']['address_components'][5]['long_name']
        zip_code = response['result']['address_components'][7]['long_name']
        full_address = response['result']['formatted_address']
        
        #Putting new values in Hospital DF
        fix_hospitals.loc[index,"Latitude"] = lat
        fix_hospitals.loc[index,"Longtitude"] = long
        fix_hospitals.loc[index,"Street Address"] = address
        fix_hospitals.loc[index,"City"] = city
        fix_hospitals.loc[index,"County"] = county
        fix_hospitals.loc[index,"State"] = state
        fix_hospitals.loc[index,"Zip Code"] = zip_code
        fix_hospitals.loc[index,"Full Address"] = full_address
        
    except:
        pass

C:\Users\kledesma\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\kledesma\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\kledesma\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [67]:
#Correct the bad address data for bad hospitals by shifting indices in the for loop
# fix_hospitals["Latitude"] = ''
# fix_hospitals["Longtitude"] = ''
# fix_hospitals["Street Address"] = ''
# fix_hospitals["City"] = ''
# fix_hospitals["County"] = ''
# fix_hospitals["State"] = ''
# fix_hospitals["Zip Code"] = ''
# fix_hospitals["Full Address"] = ''


params = {"key" : g_key
         }


url = "https://maps.googleapis.com/maps/api/place/details/json"
params["place_id"] = 'ChIJHSV3CO7aWIgRzsq072J_i_c'
response = requests.get(url, params=params).json()
#         lat = response['result']['geometry']['location']['lat']
#         long = response['result']['geometry']['location']['lng']
#         street_number = response['result']['address_components'][0]['long_name']
#         street_name = response['result']['address_components'][1]['long_name']
#         address = f"{street_number} {street_name}"


#         city = response['result']['address_components'][2]['long_name']
#         county = response['result']['address_components'][3]['long_name']
#         state = response['result']['address_components'][4]['long_name']
#         zip_code = response['result']['address_components'][6]['long_name']
#         full_address = response['result']['formatted_address']
        
        #Putting new values in Hospital DF
#         fix_hospitals.loc[index,"Latitude"] = lat
#         fix_hospitals.loc[index,"Longtitude"] = long
#         fix_hospitals.loc[index,"Street Address"] = address
#         fix_hospitals.loc[index,"City"] = city
#         fix_hospitals.loc[index,"County"] = county
#         fix_hospitals.loc[index,"State"] = state
#         fix_hospitals.loc[index,"Zip Code"] = zip_code
#         fix_hospitals.loc[index,"Full Address"] = full_address
        

pprint(response)

{'html_attributions': [],
 'result': {'address_components': [{'long_name': 'Clayton',
                                    'short_name': 'Clayton',
                                    'types': ['locality', 'political']},
                                   {'long_name': 'Rabun County',
                                    'short_name': 'Rabun County',
                                    'types': ['administrative_area_level_2',
                                              'political']},
                                   {'long_name': 'Georgia',
                                    'short_name': 'GA',
                                    'types': ['administrative_area_level_1',
                                              'political']},
                                   {'long_name': 'United States',
                                    'short_name': 'US',
                                    'types': ['country', 'political']},
                                   {'long_name': '30525',
     

In [61]:
#Verifying data is fixed for fix_hospitals df
fix_hospitals

,Hospital name,City,County,Acute Care Beds,Traumacenterdesignation,COTH[2]TeachingHospital,YearFounded,Affiliation,Notes,Place ID,Latitude,Longtitude,Street Address,State,Zip Code,Full Address
6,Augusta University Children's Hospital of Georgia,Augusta,Richmond County,154,NaN,No,NaN,Augusta University,NaN,ChIJoyV_F0jM-YgRqAe60-Abauk,33.4711,-81.9878,Harper Street Medical District,Georgia,30912,"1446 Harper St, Augusta, GA 30912, USA"
7,Augusta University Medical Center,Augusta,Richmond County,478,Level I,Yes,NaN,Augusta University,NaN,ChIJyX-W70nM-YgRtaRCmY78n_g,33.4713,-81.9879,15th Street Medical District,Georgia,30912,"1120 15th St, Augusta, GA 30912, USA"
12,Candler Hospital,Savannah,Chatham County,384,NaN,NaN,1804,SJCHS,"Oldest hospital in Georgia, and second oldest ...",ChIJpb2fD5N1-4gRM-PXE90FV6c,32.0278,-81.1001,Reynolds Street Midtown,Georgia,31405,"5353 Reynolds St, Savannah, GA 31405, USA"
17,Children's Healthcare of Atlanta Hughes Spalding,Atlanta,Fulton County,NaN,NaN,Yes,1952,CHOA,NaN,ChIJE0RlP4kD9YgRf-FdJvSfWXw,33.7539,-84.382,Jesse Hill Junior Drive Southeast Downtown Atl...,Georgia,30303,"35 Jesse Hill Jr Dr SE, Atlanta, GA 30303, USA"
18,Children's Healthcare of Atlanta Scottish Rite,Atlanta,Fulton County,NaN,Level II Peds,Yes,1915,CHOA,NaN,ChIJV-6vqjQJ9YgRhvQE1FyLn-g,33.9062,-84.3532,Johnson Ferry Road Northeast Perimeter Center,Georgia,30342,"1001 Johnson Ferry Rd NE, Atlanta, GA 30342, USA"
30,Eastside Medical Center,Peachtree Corners,Gwinnett County,249,NaN,NaN,NaN,HCA,NaN,ChIJaTUBHwOh9YgRoHi7L2zoz4E,33.964,-84.2259,5707 Peachtree Parkway,Georgia,30092,"5707 Peachtree Pkwy #100, Peachtree Corners, G..."
37,Emory Saint Joseph's Hospital,Atlanta,Fulton County,276,NaN,NaN,1880,Emory,Formerly Saint Joseph's Hospital of Atlanta,ChIJW3epPTQJ9YgRxf9a5U20qNg,33.9084,-84.3499,Peachtree Dunwoody Road Perimeter Center,Georgia,30342,"5665 Peachtree Dunwoody Rd, Atlanta, GA 30342,..."
39,Emory University Hospital Midtown,Atlanta,Fulton County,511,NaN,Yes,1908,Emory,Formerly Crawford W. Long Memorial Hospital,ChIJ42AWonoE9YgRonLLiSdYxtA,33.7687,-84.3864,Peachtree Street Northeast SoNo,Georgia,30308,"550 Peachtree St NE, Atlanta, GA 30308, USA"
43,Fannin Regional Hospital,Blue Ridge,Fannin County,50,NaN,NaN,NaN,â,NaN,ChIJN7SfeV2vX4gRu5N9mgngcXo,34.9177,-84.3743,2855 Old Highway 5,Georgia,30513,"2855 Old Hwy 5 North, Blue Ridge, GA 30513, USA"
47,Grady Memorial Hospital,Atlanta,Fulton County,961,Level I,Yes,1892,â,NaN,ChIJ6Rc8z44D9YgR28vCVjqbUGw,33.7518,-84.382,Jesse Hill Junior Drive Southeast Downtown Atl...,Georgia,30303,"80 Jesse Hill Jr Dr SE, Atlanta, GA 30303, USA"


In [62]:
#Concatenating the two DF's back together and printing, doing some checks
cleaned_hospitals = pd.concat([correct_hospitals, fix_hospitals])

cleaned_hospitals["State"].unique()
cleaned_hospitals["City"].unique()
cleaned_hospitals["County"].unique()

array(['Gordon County', 'Murray County', 'Thomas County', 'Brooks County',
       'Grady County', 'Mitchell County', 'Bibb County',
       'Bleckley County', 'Coweta County', 'Candler County',
       'Bartow County', 'Towns County', 'Catoosa County', 'DeKalb County',
       'Clinch County', 'Coffee County', 'Colquitt County',
       'Crisp County', 'Richmond County', 'Dodge County',
       'Seminole County', 'Ben Hill County', 'Bulloch County',
       'Effingham County', 'Elbert County', 'Emanuel County',
       'Fulton County', 'Evans County', 'Laurens County', 'Macon County',
       'Floyd County', 'Polk County', 'Whitfield County',
       'Haralson County', 'Houston County', 'Irwin County',
       'Jasper County', 'Jeff Davis County', 'Jenkins County',
       'Liberty County', 'Early County', 'Toombs County', 'Peach County',
       'Decatur County', 'Ware County', 'Miller County', 'Monroe County',
       'Morgan County', 'Baldwin County', 'Hall County', 'Barrow County',
       'Lump

In [64]:
#Re-ordering the columns, dropping irrelevant columns
del cleaned_hospitals['Traumacenterdesignation']
del cleaned_hospitals['COTH[2]TeachingHospital']

In [65]:
cleaned_hospitals

,Hospital name,City,County,Acute Care Beds,YearFounded,Affiliation,Notes,Place ID,Latitude,Longtitude,Street Address,State,Zip Code,Full Address
0,AdventHealth Gordon,Calhoun,Gordon County,69,NaN,AdventHealth,Formerly Gordon Hospital,ChIJfVjpjer9X4gRGK7NwGG_tbg,34.5102,-84.9263,1035 Red Bud Road Northeast,Georgia,30701,"1035 Red Bud Rd NE, Calhoun, GA 30701, USA"
1,AdventHealth Murray,Chatsworth,Murray County,29,NaN,AdventHealth,Formerly Murray Medical Center,ChIJ-U0scjndX4gRG-FoXnNaHI4,34.78,-84.783,707 Old Dalton Ellijay Road,Georgia,30705,"707 Old Dalton Ellijay Rd, Chatsworth, GA 3070..."
2,Archbold Memorial Hospital,Thomasville,Thomas County,264,1925,Archbold,NaN,ChIJsxblg8q17YgR8XXHSBDuObQ,30.8252,-83.9721,915 Gordon Avenue,Georgia,31792,"915 Gordon Ave, Thomasville, GA 31792, USA"
3,Archbold - Brooks County Hospital,Quitman,Brooks County,25,NaN,Archbold,NaN,ChIJiyuklBp_7ogRmoGX5XNztMM,30.795,-83.5591,903 North Court Street,Georgia,31643,"903 N Court St, Quitman, GA 31643, USA"
4,Archbold - Grady General Hospital,Cairo,Grady County,60,1960,Archbold,NaN,ChIJL2eo1Vup7YgRbnlST-utsVE,30.8638,-84.1991,1155 5th Street Southeast,Georgia,39828,"1155 5th St SE, Cairo, GA 39828, USA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,St. Francis Hospital,Decatur,DeKalb County,360,1950,Emory,NaN,ChIJrzBX-3gH9YgRpeQf21LpXF4,33.7958,-84.277,Dekalb Industrial Way Greater Valley Brook,Georgia,30033,"625 Dekalb Industrial Way, Decatur, GA 30033, USA"
110,St. Joseph's Hospital,Atlanta,Fulton County,330,1875,SJCHS,NaN,ChIJW3epPTQJ9YgRxf9a5U20qNg,33.9084,-84.3499,Peachtree Dunwoody Road Perimeter Center,Georgia,30342,"5665 Peachtree Dunwoody Rd, Atlanta, GA 30342,..."
111,St. Mary's Hospital,Atlanta,Fulton County,170,NaN,St. Mary's,NaN,ChIJW3epPTQJ9YgRxf9a5U20qNg,33.9084,-84.3499,Peachtree Dunwoody Road Perimeter Center,Georgia,30342,"5665 Peachtree Dunwoody Rd, Atlanta, GA 30342,..."
126,Wellstar Atlanta Medical Center,Atlanta,Fulton County,460,1901,WellStar,NaN,ChIJH9RgbwsE9YgRCefukbDOojs,33.7629,-84.373,Parkway Drive Northeast Old Fourth Ward,Georgia,30312,"303 Parkway Dr NE, Atlanta, GA 30312, USA"


In [66]:
#write to csv
hospital_data = cleaned_hospitals.to_csv(r"Cleaned_Hospitalsv2.csv")

In [17]:
cleaned_hospitals.dtypes

Hospital name      object
City               object
County             object
Acute Care Beds    object
YearFounded        object
Affiliation        object
Notes              object
Place ID           object
Latitude           object
Longtitude         object
Street Address     object
State              object
Zip Code           object
Full Address       object
dtype: object

In [80]:
#Made edits in excel, pulling back in
hospital = pd.read_csv("Cleaned_Hospitalsv2.csv")
hospital.head()

,Unnamed: 0,Hospital name,City,County,Acute Care Beds,YearFounded,Affiliation,Notes,Place ID,Latitude,Longtitude,Street Address,State,Zip Code,Full Address
0,0,AdventHealth Gordon,Calhoun,Gordon County,69,NaN,AdventHealth,Formerly Gordon Hospital,ChIJfVjpjer9X4gRGK7NwGG_tbg,34.510205,-84.926304,1035 Red Bud Road Northeast,Georgia,30701.0,"1035 Red Bud Rd NE, Calhoun, GA 30701, USA"
1,1,AdventHealth Murray,Chatsworth,Murray County,29,NaN,AdventHealth,Formerly Murray Medical Center,ChIJ-U0scjndX4gRG-FoXnNaHI4,34.779956,-84.783038,707 Old Dalton Ellijay Road,Georgia,30705.0,"707 Old Dalton Ellijay Rd, Chatsworth, GA 3070..."
2,2,Archbold Memorial Hospital,Thomasville,Thomas County,264,1925,Archbold,NaN,ChIJsxblg8q17YgR8XXHSBDuObQ,30.825169,-83.972128,915 Gordon Avenue,Georgia,31792.0,"915 Gordon Ave, Thomasville, GA 31792, USA"
3,3,Archbold - Brooks County Hospital,Quitman,Brooks County,25,NaN,Archbold,NaN,ChIJiyuklBp_7ogRmoGX5XNztMM,30.795039,-83.559146,903 North Court Street,Georgia,31643.0,"903 N Court St, Quitman, GA 31643, USA"
4,4,Archbold - Grady General Hospital,Cairo,Grady County,60,1960,Archbold,NaN,ChIJL2eo1Vup7YgRbnlST-utsVE,30.863815,-84.199066,1155 5th Street Southeast,Georgia,39828.0,"1155 5th St SE, Cairo, GA 39828, USA"


In [19]:
#hospital_data = cleaned_hospitals.fillna("", inplace=False)
#cleaned_hospitals["Latitude"].astype(str).astype(int)
#ospital_data = cleaned_hospitals.loc[cleaned_hospitals["Latitude"] >0]


ValueError: invalid literal for int() with base 10: '34.5102054'

In [82]:
hospital.head()

,Unnamed: 0,Hospital name,City,County,Acute Care Beds,YearFounded,Affiliation,Notes,Place ID,Latitude,Longtitude,Street Address,State,Zip Code,Full Address
0,0,AdventHealth Gordon,Calhoun,Gordon County,69,NaN,AdventHealth,Formerly Gordon Hospital,ChIJfVjpjer9X4gRGK7NwGG_tbg,34.510205,-84.926304,1035 Red Bud Road Northeast,Georgia,30701.0,"1035 Red Bud Rd NE, Calhoun, GA 30701, USA"
1,1,AdventHealth Murray,Chatsworth,Murray County,29,NaN,AdventHealth,Formerly Murray Medical Center,ChIJ-U0scjndX4gRG-FoXnNaHI4,34.779956,-84.783038,707 Old Dalton Ellijay Road,Georgia,30705.0,"707 Old Dalton Ellijay Rd, Chatsworth, GA 3070..."
2,2,Archbold Memorial Hospital,Thomasville,Thomas County,264,1925,Archbold,NaN,ChIJsxblg8q17YgR8XXHSBDuObQ,30.825169,-83.972128,915 Gordon Avenue,Georgia,31792.0,"915 Gordon Ave, Thomasville, GA 31792, USA"
3,3,Archbold - Brooks County Hospital,Quitman,Brooks County,25,NaN,Archbold,NaN,ChIJiyuklBp_7ogRmoGX5XNztMM,30.795039,-83.559146,903 North Court Street,Georgia,31643.0,"903 N Court St, Quitman, GA 31643, USA"
4,4,Archbold - Grady General Hospital,Cairo,Grady County,60,1960,Archbold,NaN,ChIJL2eo1Vup7YgRbnlST-utsVE,30.863815,-84.199066,1155 5th Street Southeast,Georgia,39828.0,"1155 5th St SE, Cairo, GA 39828, USA"


In [83]:
hospitallist= hospital["Hospital name"].tolist()


fig = gmaps.figure()

locations = hospital[['Latitude', 'Longtitude']]


#Creating heat layer
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Name: {city}" for city in hospitallist])

fig.add_layer(markers)

fig



InvalidPointException: nan is not a valid latitude. Latitudes must lie between -90 and 90.

In [84]:
hospital.head()

,Unnamed: 0,Hospital name,City,County,Acute Care Beds,YearFounded,Affiliation,Notes,Place ID,Latitude,Longtitude,Street Address,State,Zip Code,Full Address
0,0,AdventHealth Gordon,Calhoun,Gordon County,69,NaN,AdventHealth,Formerly Gordon Hospital,ChIJfVjpjer9X4gRGK7NwGG_tbg,34.510205,-84.926304,1035 Red Bud Road Northeast,Georgia,30701.0,"1035 Red Bud Rd NE, Calhoun, GA 30701, USA"
1,1,AdventHealth Murray,Chatsworth,Murray County,29,NaN,AdventHealth,Formerly Murray Medical Center,ChIJ-U0scjndX4gRG-FoXnNaHI4,34.779956,-84.783038,707 Old Dalton Ellijay Road,Georgia,30705.0,"707 Old Dalton Ellijay Rd, Chatsworth, GA 3070..."
2,2,Archbold Memorial Hospital,Thomasville,Thomas County,264,1925,Archbold,NaN,ChIJsxblg8q17YgR8XXHSBDuObQ,30.825169,-83.972128,915 Gordon Avenue,Georgia,31792.0,"915 Gordon Ave, Thomasville, GA 31792, USA"
3,3,Archbold - Brooks County Hospital,Quitman,Brooks County,25,NaN,Archbold,NaN,ChIJiyuklBp_7ogRmoGX5XNztMM,30.795039,-83.559146,903 North Court Street,Georgia,31643.0,"903 N Court St, Quitman, GA 31643, USA"
4,4,Archbold - Grady General Hospital,Cairo,Grady County,60,1960,Archbold,NaN,ChIJL2eo1Vup7YgRbnlST-utsVE,30.863815,-84.199066,1155 5th Street Southeast,Georgia,39828.0,"1155 5th St SE, Cairo, GA 39828, USA"


In [85]:
hospital["Acute Care Beds"].fillna("0", inplace=True)

In [86]:
hospital['Acute Care Beds'] = hospital['Acute Care Beds'].astype(int)
hospital

ValueError: invalid literal for int() with base 10: '25[citation needed]'

In [87]:
hospital_group = hospital.groupby("City")

summary = round(hospital_group['Acute Care Beds'].agg(['mean','sum','median','var','std','sem']), 2)

summary

DataError: No numeric types to aggregate

In [88]:
del hospital['Unnamed: 0']
hospital.head()

,Hospital name,City,County,Acute Care Beds,YearFounded,Affiliation,Notes,Place ID,Latitude,Longtitude,Street Address,State,Zip Code,Full Address
0,AdventHealth Gordon,Calhoun,Gordon County,69,NaN,AdventHealth,Formerly Gordon Hospital,ChIJfVjpjer9X4gRGK7NwGG_tbg,34.510205,-84.926304,1035 Red Bud Road Northeast,Georgia,30701.0,"1035 Red Bud Rd NE, Calhoun, GA 30701, USA"
1,AdventHealth Murray,Chatsworth,Murray County,29,NaN,AdventHealth,Formerly Murray Medical Center,ChIJ-U0scjndX4gRG-FoXnNaHI4,34.779956,-84.783038,707 Old Dalton Ellijay Road,Georgia,30705.0,"707 Old Dalton Ellijay Rd, Chatsworth, GA 3070..."
2,Archbold Memorial Hospital,Thomasville,Thomas County,264,1925,Archbold,NaN,ChIJsxblg8q17YgR8XXHSBDuObQ,30.825169,-83.972128,915 Gordon Avenue,Georgia,31792.0,"915 Gordon Ave, Thomasville, GA 31792, USA"
3,Archbold - Brooks County Hospital,Quitman,Brooks County,25,NaN,Archbold,NaN,ChIJiyuklBp_7ogRmoGX5XNztMM,30.795039,-83.559146,903 North Court Street,Georgia,31643.0,"903 N Court St, Quitman, GA 31643, USA"
4,Archbold - Grady General Hospital,Cairo,Grady County,60,1960,Archbold,NaN,ChIJL2eo1Vup7YgRbnlST-utsVE,30.863815,-84.199066,1155 5th Street Southeast,Georgia,39828.0,"1155 5th St SE, Cairo, GA 39828, USA"


In [27]:
#Reading in census data, removing unnecessary columns
census_data = pd.read_csv("Census_data_v2.csv")
# del census_data['Unnamed: 0']
# del census_data['State_Code']
# del census_data['State']

In [ ]:
#Cleaning data
census_data = census_data.replace(to_replace='city',value="",regex=True)
census_data = census_data.replace(to_replace='town',value="",regex=True)
census_data = census_data.replace(to_replace='CDP',value="",regex=True)
census_data = census_data.replace(to_replace='unified government',value="",regex=True)

In [28]:
#Grouping by city then resetting index
census_group = census_data.groupby("City").mean()
census_group = census_group.reset_index()
census_group
#census = census_group("Median Income").mean()

,City,Unnamed: 0,ZIP,Latitude,Longitude,Median Income,Population
0,Abbeville,465.0,31001.0,31.972626,-83.330820,20880.0,4563.0
1,Acworth,630.0,30101.5,34.075142,-84.651565,33711.5,49841.5
2,Adairsville,280.0,30103.0,34.368647,-84.924520,24570.0,16138.0
3,Adel,673.0,31620.0,31.133004,-83.427360,25632.0,10836.0
4,Adrian,733.0,31002.0,32.549854,-82.569480,19562.0,3071.0
...,...,...,...,...,...,...,...
625,Wrens,75.0,30833.0,33.200325,-82.376250,18634.0,3065.0
626,Wrightsville,155.0,31096.0,32.717320,-82.689260,20232.0,7046.0
627,Yatesville,684.0,31097.0,32.912207,-84.148120,27000.0,1656.0
628,Young Harris,442.0,30582.0,34.954638,-83.864840,15867.0,5038.0


In [ ]:
#Trying to trim whitespace but not having luck
census_group['City'] = census_group['City'].str.strip
census_group

In [36]:
#Trying to import the file manually cleaned that has no more trailing whitespaces
census_data2 = pd.read_csv("Census_data_v2.csv")
census_group = census_data2.groupby("City").mean()
census_group = census_group.reset_index()
del census_group['Unnamed: 0']


In [37]:
census_group

,City,ZIP,Latitude,Longitude,Median Income,Population
0,Abbeville,31001.0,31.972626,-83.330820,20880.0,4563.0
1,Acworth,30101.5,34.075142,-84.651565,33711.5,49841.5
2,Adairsville,30103.0,34.368647,-84.924520,24570.0,16138.0
3,Adel,31620.0,31.133004,-83.427360,25632.0,10836.0
4,Adrian,31002.0,32.549854,-82.569480,19562.0,3071.0
...,...,...,...,...,...,...
625,Wrens,30833.0,33.200325,-82.376250,18634.0,3065.0
626,Wrightsville,31096.0,32.717320,-82.689260,20232.0,7046.0
627,Yatesville,31097.0,32.912207,-84.148120,27000.0,1656.0
628,Young Harris,30582.0,34.954638,-83.864840,15867.0,5038.0


In [90]:
hospital

,Hospital name,City,County,Acute Care Beds,YearFounded,Affiliation,Notes,Place ID,Latitude,Longtitude,Street Address,State,Zip Code,Full Address
0,AdventHealth Gordon,Calhoun,Gordon County,69,NaN,AdventHealth,Formerly Gordon Hospital,ChIJfVjpjer9X4gRGK7NwGG_tbg,34.510205,-84.926304,1035 Red Bud Road Northeast,Georgia,30701.0,"1035 Red Bud Rd NE, Calhoun, GA 30701, USA"
1,AdventHealth Murray,Chatsworth,Murray County,29,NaN,AdventHealth,Formerly Murray Medical Center,ChIJ-U0scjndX4gRG-FoXnNaHI4,34.779956,-84.783038,707 Old Dalton Ellijay Road,Georgia,30705.0,"707 Old Dalton Ellijay Rd, Chatsworth, GA 3070..."
2,Archbold Memorial Hospital,Thomasville,Thomas County,264,1925,Archbold,NaN,ChIJsxblg8q17YgR8XXHSBDuObQ,30.825169,-83.972128,915 Gordon Avenue,Georgia,31792.0,"915 Gordon Ave, Thomasville, GA 31792, USA"
3,Archbold - Brooks County Hospital,Quitman,Brooks County,25,NaN,Archbold,NaN,ChIJiyuklBp_7ogRmoGX5XNztMM,30.795039,-83.559146,903 North Court Street,Georgia,31643.0,"903 N Court St, Quitman, GA 31643, USA"
4,Archbold - Grady General Hospital,Cairo,Grady County,60,1960,Archbold,NaN,ChIJL2eo1Vup7YgRbnlST-utsVE,30.863815,-84.199066,1155 5th Street Southeast,Georgia,39828.0,"1155 5th St SE, Cairo, GA 39828, USA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,St. Francis Hospital,Decatur,DeKalb County,360,1950,Emory,NaN,ChIJrzBX-3gH9YgRpeQf21LpXF4,33.795761,-84.276954,Dekalb Industrial Way Greater Valley Brook,Georgia,30033.0,"625 Dekalb Industrial Way, Decatur, GA 30033, USA"
133,St. Joseph's Hospital,Atlanta,Fulton County,330,1875,SJCHS,NaN,ChIJW3epPTQJ9YgRxf9a5U20qNg,33.908422,-84.349851,Peachtree Dunwoody Road Perimeter Center,Georgia,30342.0,"5665 Peachtree Dunwoody Rd, Atlanta, GA 30342,..."
134,St. Mary's Hospital,Atlanta,Fulton County,170,NaN,St. Mary's,NaN,ChIJW3epPTQJ9YgRxf9a5U20qNg,33.908422,-84.349851,Peachtree Dunwoody Road Perimeter Center,Georgia,30342.0,"5665 Peachtree Dunwoody Rd, Atlanta, GA 30342,..."
135,Wellstar Atlanta Medical Center,Atlanta,Fulton County,460,1901,WellStar,NaN,ChIJH9RgbwsE9YgRCefukbDOojs,33.762907,-84.372961,Parkway Drive Northeast Old Fourth Ward,Georgia,30312.0,"303 Parkway Dr NE, Atlanta, GA 30312, USA"


In [89]:
hospital_fix = hospital.loc[(hospital['State'] != 'Georgia')]
hospital_fix

,Hospital name,City,County,Acute Care Beds,YearFounded,Affiliation,Notes,Place ID,Latitude,Longtitude,Street Address,State,Zip Code,Full Address
126,Jefferson Hospital,Philadelphia,Philadelphia County,37,NaN,â,NaN,ChIJ-W1LT-3HxokRD_nmm7DDoU0,39.949562,-75.158032,South 11th Street Center City East,Pennsylvania,19107.0,"111 S 11th St, Philadelphia, PA 19107, USA"
128,Mountain Lakes Medical Center,NaN,NaN,0,NaN,â,NaN,ChIJHSV3CO7aWIgRzsq072J_i_c,NaN,NaN,NaN,NaN,NaN,NaN
131,Piedmont Henry Hospital,NaN,NaN,277,NaN,Piedmont,NaN,ChIJBduprw5X9IgRygZRl0TyxOs,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
#still not having luck merging, even with the trailing whitespaces removed...
merge = pd.merge(hospital, census_group, how = 'left', on = 'City')
merge

,Hospital name,City,County,Acute Care Beds,YearFounded,Affiliation,Notes,Place ID,Latitude_x,Longtitude,Street Address,State,Zip Code,Full Address,ZIP,Latitude_y,Longitude,Median Income,Population
0,AdventHealth Gordon,Calhoun,Gordon County,69,NaN,AdventHealth,Formerly Gordon Hospital,ChIJfVjpjer9X4gRGK7NwGG_tbg,34.510205,-84.926304,1035 Red Bud Road Northeast,Georgia,30701.0,"1035 Red Bud Rd NE, Calhoun, GA 30701, USA",30702.000000,34.487833,-84.850071,2.411000e+04,40820.000000
1,AdventHealth Murray,Chatsworth,Murray County,29,NaN,AdventHealth,Formerly Murray Medical Center,ChIJ-U0scjndX4gRG-FoXnNaHI4,34.779956,-84.783038,707 Old Dalton Ellijay Road,Georgia,30705.0,"707 Old Dalton Ellijay Rd, Chatsworth, GA 3070...",30705.000000,34.753129,-84.775020,2.445700e+04,33460.000000
2,Archbold Memorial Hospital,Thomasville,Thomas County,264,1925,Archbold,NaN,ChIJsxblg8q17YgR8XXHSBDuObQ,30.825169,-83.972128,915 Gordon Avenue,Georgia,31792.0,"915 Gordon Ave, Thomasville, GA 31792, USA",31776.500000,30.862504,-83.928812,2.249825e+04,20703.750000
3,Archbold - Brooks County Hospital,Quitman,Brooks County,25,NaN,Archbold,NaN,ChIJiyuklBp_7ogRmoGX5XNztMM,30.795039,-83.559146,903 North Court Street,Georgia,31643.0,"903 N Court St, Quitman, GA 31643, USA",31643.000000,30.783639,-83.553430,2.464400e+04,9062.000000
4,Archbold - Grady General Hospital,Cairo,Grady County,60,1960,Archbold,NaN,ChIJL2eo1Vup7YgRbnlST-utsVE,30.863815,-84.199066,1155 5th Street Southeast,Georgia,39828.0,"1155 5th St SE, Cairo, GA 39828, USA",31728.000000,30.870390,-84.213430,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,St. Francis Hospital,Decatur,DeKalb County,360,1950,Emory,NaN,ChIJrzBX-3gH9YgRpeQf21LpXF4,33.795761,-84.276954,Dekalb Industrial Way Greater Valley Brook,Georgia,30033.0,"625 Dekalb Industrial Way, Decatur, GA 30033, USA",30033.500000,33.802007,-84.190947,3.340138e+04,37842.875000
133,St. Joseph's Hospital,Atlanta,Fulton County,330,1875,SJCHS,NaN,ChIJW3epPTQJ9YgRxf9a5U20qNg,33.908422,-84.349851,Peachtree Dunwoody Road Perimeter Center,Georgia,30342.0,"5665 Peachtree Dunwoody Rd, Atlanta, GA 30342,...",30576.765766,33.828459,-84.402366,-2.194366e+07,25919.934066
134,St. Mary's Hospital,Atlanta,Fulton County,170,NaN,St. Mary's,NaN,ChIJW3epPTQJ9YgRxf9a5U20qNg,33.908422,-84.349851,Peachtree Dunwoody Road Perimeter Center,Georgia,30342.0,"5665 Peachtree Dunwoody Rd, Atlanta, GA 30342,...",30576.765766,33.828459,-84.402366,-2.194366e+07,25919.934066
135,Wellstar Atlanta Medical Center,Atlanta,Fulton County,460,1901,WellStar,NaN,ChIJH9RgbwsE9YgRCefukbDOojs,33.762907,-84.372961,Parkway Drive Northeast Old Fourth Ward,Georgia,30312.0,"303 Parkway Dr NE, Atlanta, GA 30312, USA",30576.765766,33.828459,-84.402366,-2.194366e+07,25919.934066


In [119]:
merge['Joined'] = merge['Median Income'].isnull()
unjoined = merge.loc[merge['Joined'] == True]

unjoined = unjoined.loc[unjoined['State'] == 'Georgia']
unjoined

,Hospital name,City,County,Acute Care Beds,YearFounded,Affiliation,Notes,Place ID,Latitude_x,Longtitude,Street Address,State,Zip Code,Full Address,ZIP,Latitude_y,Longitude,Median Income,Population,Joined
4,Archbold - Grady General Hospital,Cairo,Grady County,60,1960,Archbold,NaN,ChIJL2eo1Vup7YgRbnlST-utsVE,30.863815,-84.199066,1155 5th Street Southeast,Georgia,39828.0,"1155 5th St SE, Cairo, GA 39828, USA",31728.0,30.870390,-84.213430,NaN,NaN,True
22,Donalsonville Hospital,Donalsonville,Seminole County,65,1919,â,NaN,ChIJDROJ95Qz7YgR4AS_uCutkDI,31.050927,-84.880666,102 Hospital Cir,Georgia,39845.0,"102 Hospital Cir, Donalsonville, GA 39845, USA",31745.0,30.974364,-84.877030,NaN,NaN,True
30,Emory Johns Creek Hospital,Johns Creek,Fulton County,118,2007,Emory,NaN,ChIJ8wevZy-Z9YgR5mpDmsMDM48,34.066497,-84.176237,6325 Hospital Parkway,Georgia,30097.0,"6325 Hospital Pkwy, Johns Creek, GA 30097, USA",NaN,NaN,NaN,NaN,NaN,True
47,LifeBrite Community Hospital of Early,Blakely,Early County,25,NaN,LifeBrite,Formerly Pioneer Community Hospital of Early,ChIJrc5daj-tkogRy9bc9VL33QQ,31.373318,-84.945705,11740 Columbia Street,Georgia,39823.0,"11740 Columbia Street, Blakely, GA 39823, USA",31723.0,31.334337,-84.948990,NaN,NaN,True
51,Memorial Hospital and Manor,Bainbridge,Decatur County,80/107,1960,â,NaN,ChIJy4AyxxAZ7YgRUTmGeEmLfp4,30.902716,-84.552460,1500 East Shotwell Street,Georgia,39819.0,"1500 E Shotwell St, Bainbridge, GA 39819, USA",31717.5,30.893194,-84.579780,NaN,NaN,True
53,Miller County Hospital,Colquitt,Miller County,25,1957,â,NaN,ChIJ03tgHp9H7YgRzI6aVejbskA,31.175429,-84.735244,209 North Cuthbert Street,Georgia,39837.0,"209 N Cuthbert St, Colquitt, GA 39837, USA",31737.0,31.152253,-84.696530,NaN,NaN,True
87,Southeast Georgia Health System Camden,St. Marys,Camden County,40,NaN,SGHS,NaN,ChIJU6IsZ5cE5YgRaTwbgslwgUw,30.780916,-81.615147,2000 Dan Proctor Drive,Georgia,31558.0,"2000 Dan Proctor Dr, St Marys, GA 31558, USA",NaN,NaN,NaN,NaN,NaN,True
91,Southwest Georgia Regional Medical Center,Cuthbert,Randolph County,105,1916/1947,â,Formerly Patterson Hospital,ChIJwYwPs8Gw8ogRsx59PxkaP74,31.775420,-84.793018,361 Randolph Street,Georgia,39840.0,"361 Randolph St, Cuthbert, GA 39840, USA",31740.0,31.781815,-84.764440,NaN,NaN,True
113,WellStar West Georgia Medical Center,LaGrange,Troup County,542,1937,WellStar,NaN,ChIJ40ZJ5vtWi4gRK6PnDb-BAPU,33.030940,-85.058637,1514 Vernon Road,Georgia,30240.0,"1514 Vernon Rd, LaGrange, GA 30240, USA",NaN,NaN,NaN,NaN,NaN,True
120,Eastside Medical Center,Peachtree Corners,Gwinnett County,249,NaN,HCA,NaN,ChIJaTUBHwOh9YgRoHi7L2zoz4E,33.964011,-84.225939,5707 Peachtree Parkway,Georgia,30092.0,"5707 Peachtree Pkwy #100, Peachtree Corners, G...",NaN,NaN,NaN,NaN,NaN,True


In [124]:
#attempting to join based on zip code instead of city
merge2 = pd.merge(unjoined, census_group,how = 'inner', left_on = 'Zip Code', right_on = 'ZIP')
merge2


,Hospital name,City_x,County,Acute Care Beds,YearFounded,Affiliation,Notes,Place ID,Latitude_x,Longtitude,...,Longitude_x,Median Income_x,Population_x,Joined,City_y,ZIP_y,Latitude,Longitude_y,Median Income_y,Population_y
0,Southeast Georgia Health System Camden,St. Marys,Camden County,40,NaN,SGHS,NaN,ChIJU6IsZ5cE5YgRaTwbgslwgUw,30.780916,-81.615147,...,NaN,NaN,NaN,True,Saint Marys,31558.0,30.761081,-81.56912,29862.0,21307.0


In [120]:
hospital['City'].unique()

array(['Calhoun', 'Chatsworth', 'Thomasville', 'Quitman', 'Cairo',
       'Camilla', 'Macon', 'Cochran', 'Newnan', 'Metter', 'Cartersville',
       'Hiawassee', 'Fort Oglethorpe', 'Atlanta', 'Homerville', 'Douglas',
       'Moultrie', 'Cordele', 'Augusta', 'Eastman', 'Donalsonville',
       'Fitzgerald', 'Statesboro', 'Springfield', 'Elberton',
       'Swainsboro', 'Decatur', 'Lithonia', 'Johns Creek', 'Tucker',
       'Claxton', 'Dublin', 'Montezuma', 'Rome', 'Cedartown', 'Dalton',
       'Bremen', 'Warner Robins', 'Ocilla', 'Monticello', 'Hazlehurst',
       'Millen', 'Hinesville', 'Blakely', 'Vidalia', 'Byron',
       'Bainbridge', 'Waycross', 'Colquitt', 'Forsyth', 'Madison',
       'Milledgeville', 'Gainesville', 'Winder', 'Braselton', 'Dahlonega',
       'Commerce', 'Canton', 'Duluth', 'Cumming', 'Lawrenceville',
       'Sylvania', 'Reidsville', 'Albany', 'Americus', 'Sylvester',
       'Athens', 'Columbus', 'Fayetteville', 'Jasper', 'Covington',
       'Conyers', 'Monroe', 'Eato

In [127]:
unjoined_cities = unjoined['City'].unique()
city_map = pd.DataFrame(unjoined_cities)
city_map.to_csv(r"city_map.csv")

In [128]:
census_cities = census_data['City'].unique()
census_cities = pd.DataFrame(census_cities)
census_cities.to_csv(r"census_cities.csv")

In [56]:
# hospital['Zip Code'].astype(float)
# hospital.dtypes
hospital.loc[hospital['Zip Code'] == 'United States']

,Hospital name,City,County,Acute Care Beds,YearFounded,Affiliation,Notes,Place ID,Latitude,Longtitude,Street Address,State,Zip Code,Full Address,Unnamed: 15
115,Augusta University Children's Hospital of Georgia,Medical District,Augusta,154,NaN,Augusta University,NaN,ChIJoyV_F0jM-YgRqAe60-Abauk,33.471138,-81.987774,1446 Harper Street,Richmond County,United States,"1446 Harper St, Augusta, GA 30912, USA",NaN
116,Augusta University Medical Center,Medical District,Augusta,478,NaN,Augusta University,NaN,ChIJyX-W70nM-YgRtaRCmY78n_g,33.471349,-81.987897,1120 15th Street,Richmond County,United States,"1120 15th St, Augusta, GA 30912, USA",NaN
117,Candler Hospital,Midtown,Savannah,384,1804,SJCHS,"Oldest hospital in Georgia, and second oldest ...",ChIJpb2fD5N1-4gRM-PXE90FV6c,32.027829,-81.100126,5353 Reynolds Street,Chatham County,United States,"5353 Reynolds St, Savannah, GA 31405, USA",NaN
118,Children's Healthcare of Atlanta Hughes Spalding,Downtown Atlanta,Atlanta,0,1952,CHOA,NaN,ChIJE0RlP4kD9YgRf-FdJvSfWXw,33.753901,-84.382009,35 Jesse Hill Junior Drive Southeast,Fulton County,United States,"35 Jesse Hill Jr Dr SE, Atlanta, GA 30303, USA",NaN
119,Children's Healthcare of Atlanta Scottish Rite,Perimeter Center,Atlanta,0,1915,CHOA,NaN,ChIJV-6vqjQJ9YgRhvQE1FyLn-g,33.906220,-84.353199,1001 Johnson Ferry Road Northeast,Fulton County,United States,"1001 Johnson Ferry Rd NE, Atlanta, GA 30342, USA",NaN
120,Eastside Medical Center,Peachtree Parkway,Peachtree Corners,249,NaN,HCA,NaN,ChIJaTUBHwOh9YgRoHi7L2zoz4E,33.964011,-84.225939,100 5707,Gwinnett County,United States,"5707 Peachtree Pkwy #100, Peachtree Corners, G...",NaN
121,Emory Saint Joseph's Hospital,Perimeter Center,Atlanta,276,1880,Emory,Formerly Saint Joseph's Hospital of Atlanta,ChIJW3epPTQJ9YgRxf9a5U20qNg,33.908422,-84.349851,5665 Peachtree Dunwoody Road,Fulton County,United States,"5665 Peachtree Dunwoody Rd, Atlanta, GA 30342,...",NaN
122,Emory University Hospital Midtown,SoNo,Atlanta,511,1908,Emory,Formerly Crawford W. Long Memorial Hospital,ChIJ42AWonoE9YgRonLLiSdYxtA,33.768660,-84.386418,550 Peachtree Street Northeast,Fulton County,United States,"550 Peachtree St NE, Atlanta, GA 30308, USA",NaN
123,Fannin Regional Hospital,Old Highway 5,Blue Ridge,50,NaN,â,NaN,ChIJN7SfeV2vX4gRu5N9mgngcXo,34.917735,-84.374330,North 2855,Fannin County,United States,"2855 Old Hwy 5 North, Blue Ridge, GA 30513, USA",NaN
124,Grady Memorial Hospital,Downtown Atlanta,Atlanta,961,1892,â,NaN,ChIJ6Rc8z44D9YgR28vCVjqbUGw,33.751785,-84.382007,80 Jesse Hill Junior Drive Southeast,Fulton County,United States,"80 Jesse Hill Jr Dr SE, Atlanta, GA 30303, USA",NaN


In [52]:
census_group.dtypes

City              object
ZIP              float64
Latitude         float64
Longitude        float64
Median Income    float64
Population       float64
dtype: object

In [109]:
zipmerge = pd.merge(hospital, census_group, how = 'left', left_on = 'Zip Code', right_on = 'ZIP')
zipmerge

,Hospital name,City_x,County,Acute Care Beds,YearFounded,Affiliation,Notes,Place ID,Latitude_x,Longtitude,Street Address,State,Zip Code,Full Address,City_y,ZIP,Latitude_y,Longitude,Median Income,Population
0,AdventHealth Gordon,Calhoun,Gordon County,69,NaN,AdventHealth,Formerly Gordon Hospital,ChIJfVjpjer9X4gRGK7NwGG_tbg,34.510205,-84.926304,1035 Red Bud Road Northeast,Georgia,30701.0,"1035 Red Bud Rd NE, Calhoun, GA 30701, USA",NaN,NaN,NaN,NaN,NaN,NaN
1,AdventHealth Murray,Chatsworth,Murray County,29,NaN,AdventHealth,Formerly Murray Medical Center,ChIJ-U0scjndX4gRG-FoXnNaHI4,34.779956,-84.783038,707 Old Dalton Ellijay Road,Georgia,30705.0,"707 Old Dalton Ellijay Rd, Chatsworth, GA 3070...",Chatsworth,30705.0,34.753129,-84.77502,24457.0,33460.0
2,Archbold Memorial Hospital,Thomasville,Thomas County,264,1925,Archbold,NaN,ChIJsxblg8q17YgR8XXHSBDuObQ,30.825169,-83.972128,915 Gordon Avenue,Georgia,31792.0,"915 Gordon Ave, Thomasville, GA 31792, USA",NaN,NaN,NaN,NaN,NaN,NaN
3,Archbold - Brooks County Hospital,Quitman,Brooks County,25,NaN,Archbold,NaN,ChIJiyuklBp_7ogRmoGX5XNztMM,30.795039,-83.559146,903 North Court Street,Georgia,31643.0,"903 N Court St, Quitman, GA 31643, USA",Quitman,31643.0,30.783639,-83.55343,24644.0,9062.0
4,Archbold - Grady General Hospital,Cairo,Grady County,60,1960,Archbold,NaN,ChIJL2eo1Vup7YgRbnlST-utsVE,30.863815,-84.199066,1155 5th Street Southeast,Georgia,39828.0,"1155 5th St SE, Cairo, GA 39828, USA",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,St. Francis Hospital,Decatur,DeKalb County,360,1950,Emory,NaN,ChIJrzBX-3gH9YgRpeQf21LpXF4,33.795761,-84.276954,Dekalb Industrial Way Greater Valley Brook,Georgia,30033.0,"625 Dekalb Industrial Way, Decatur, GA 30033, USA",NaN,NaN,NaN,NaN,NaN,NaN
135,St. Joseph's Hospital,Atlanta,Fulton County,330,1875,SJCHS,NaN,ChIJW3epPTQJ9YgRxf9a5U20qNg,33.908422,-84.349851,Peachtree Dunwoody Road Perimeter Center,Georgia,30342.0,"5665 Peachtree Dunwoody Rd, Atlanta, GA 30342,...",NaN,NaN,NaN,NaN,NaN,NaN
136,St. Mary's Hospital,Atlanta,Fulton County,170,NaN,St. Mary's,NaN,ChIJW3epPTQJ9YgRxf9a5U20qNg,33.908422,-84.349851,Peachtree Dunwoody Road Perimeter Center,Georgia,30342.0,"5665 Peachtree Dunwoody Rd, Atlanta, GA 30342,...",NaN,NaN,NaN,NaN,NaN,NaN
137,Wellstar Atlanta Medical Center,Atlanta,Fulton County,460,1901,WellStar,NaN,ChIJH9RgbwsE9YgRCefukbDOojs,33.762907,-84.372961,Parkway Drive Northeast Old Fourth Ward,Georgia,30312.0,"303 Parkway Dr NE, Atlanta, GA 30312, USA",NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
zipmerge['Joined'] = zipmerge['Median Income'].isnull()
unjoined = zipmerge.loc[zipmerge['Joined'] == True]

unjoined = unjoined.loc[unjoined['State'] == 'Georgia']
unjoined

,Hospital name,City_x,County,Acute Care Beds,YearFounded,Affiliation,Notes,Place ID,Latitude_x,Longtitude,...,State,Zip Code,Full Address,City_y,ZIP,Latitude_y,Longitude,Median Income,Population,Joined
0,AdventHealth Gordon,Calhoun,Gordon County,69,NaN,AdventHealth,Formerly Gordon Hospital,ChIJfVjpjer9X4gRGK7NwGG_tbg,34.510205,-84.926304,...,Georgia,30701.0,"1035 Red Bud Rd NE, Calhoun, GA 30701, USA",NaN,NaN,NaN,NaN,NaN,NaN,True
2,Archbold Memorial Hospital,Thomasville,Thomas County,264,1925,Archbold,NaN,ChIJsxblg8q17YgR8XXHSBDuObQ,30.825169,-83.972128,...,Georgia,31792.0,"915 Gordon Ave, Thomasville, GA 31792, USA",NaN,NaN,NaN,NaN,NaN,NaN,True
4,Archbold - Grady General Hospital,Cairo,Grady County,60,1960,Archbold,NaN,ChIJL2eo1Vup7YgRbnlST-utsVE,30.863815,-84.199066,...,Georgia,39828.0,"1155 5th St SE, Cairo, GA 39828, USA",NaN,NaN,NaN,NaN,NaN,NaN,True
6,Beverly Knight Olson Children's Hospital,Macon,Bibb County,0,NaN,Atrium/Navicent,NaN,ChIJDyOjXYf-84gREfrs3wfa6is,32.834737,-83.636547,...,Georgia,31201.0,"888 Pine St, Macon, GA 31201, USA",NaN,NaN,NaN,NaN,NaN,NaN,True
8,"Cancer Treatment Centers of America, Atlanta",Newnan,Coweta County,0,2012,CTCA,NaN,ChIJpVsvHDrG9IgRoTnJMqk9gxY,33.384602,-84.758061,...,Georgia,30265.0,"600 Celebrate Life Pkwy, Newnan, GA 30265, USA",NaN,NaN,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,St. Francis Hospital,Decatur,DeKalb County,360,1950,Emory,NaN,ChIJrzBX-3gH9YgRpeQf21LpXF4,33.795761,-84.276954,...,Georgia,30033.0,"625 Dekalb Industrial Way, Decatur, GA 30033, USA",NaN,NaN,NaN,NaN,NaN,NaN,True
135,St. Joseph's Hospital,Atlanta,Fulton County,330,1875,SJCHS,NaN,ChIJW3epPTQJ9YgRxf9a5U20qNg,33.908422,-84.349851,...,Georgia,30342.0,"5665 Peachtree Dunwoody Rd, Atlanta, GA 30342,...",NaN,NaN,NaN,NaN,NaN,NaN,True
136,St. Mary's Hospital,Atlanta,Fulton County,170,NaN,St. Mary's,NaN,ChIJW3epPTQJ9YgRxf9a5U20qNg,33.908422,-84.349851,...,Georgia,30342.0,"5665 Peachtree Dunwoody Rd, Atlanta, GA 30342,...",NaN,NaN,NaN,NaN,NaN,NaN,True
137,Wellstar Atlanta Medical Center,Atlanta,Fulton County,460,1901,WellStar,NaN,ChIJH9RgbwsE9YgRCefukbDOojs,33.762907,-84.372961,...,Georgia,30312.0,"303 Parkway Dr NE, Atlanta, GA 30312, USA",NaN,NaN,NaN,NaN,NaN,NaN,True
